<a href="https://colab.research.google.com/github/maribardella/agente-especificar-user-story/blob/main/Agente_de_Especifica%C3%A7%C3%A3o_de_User_Stories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [15]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [16]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [17]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [79]:
##########################################
# --- Agente 1: Analisador de Contexto --- #
##########################################
def agente_analisador_contexto(contexto):

    analisador_contexto = Agent(
        name="agente_analisador_contexto",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista de produto altamente experiente, com profundo conhecimento em elicitação de requisitos e compreensão de necessidades de negócio.
        Seu objetivo é interpretar o input fornecido por um Product Owner e extrair os elementos essenciais para criação de uma user story de qualidade.
        Utilize boas práticas de análise de requisitos para identificar:
        Quem é o usuário ou persona envolvida?
        Qual é a ação ou funcionalidade desejada?
        Qual é o benefício ou valor que essa ação traz para o usuário?
        Há telas, fluxos ou sistemas mencionados?
        Há informações ambíguas ou faltantes que precisam de esclarecimento?
        """,
        description="Agente que faz o entendimento sobre o produto, personas e funcionalidades.",
    )

    entrada_do_agente_analisador_contexto = f"Contexto: {contexto}"

    contexto_analisado = call_agent(analisador_contexto, entrada_do_agente_analisador_contexto)
    return contexto_analisado

In [75]:
################################################
# --- Agente 2: Gerador de User Stories --- #
################################################
def agente_gerador_user_stories(contexto, contexto_analisado):
    gerador_user_stories = Agent(
        name="agente_gerador_user_stories",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista de produtos e sabe especificar demandas como ninguém.
        Com base nas boas práticas de Product Management e metodologias ágeis, sua missão é redigir user stories claras, concisas e centradas no usuário.
        Utilize o formato padrão:
        Como [persona], quero [ação], para que [benefício]
        Certifique-se de que:
        A persona esteja clara e contextualizada
        A ação seja específica e objetiva
        O benefício represente o valor ou motivação real por trás da ação
        """,
        description="Agente que Produz as user stories no formato padrão",
    )

    entrada_do_agente_gerador_user_stories = f"Contexto:{contexto}\nContexto analisado: {contexto_analisado}"
    # Executa o agente
    user_stories = call_agent(gerador_user_stories, entrada_do_agente_gerador_user_stories)
    return user_stories

In [69]:
######################################
# --- Agente 3: Gerador de Critérios de Aceitação --- #
######################################
def agente_gerador_criterios(contexto, contexto_analisado, user_stories):
    gerador_criterios = Agent(
        name="agente_gerador_criterios",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Product Manager especializado em documentação de requisitos para times ágeis.
        Seu objetivo é transformar uma user story em critérios de aceitação claros, objetivos e testáveis, garantindo que o time de desenvolvimento e QA saiba exatamente quando considerar a entrega como "pronta".
        Siga estas boas práticas:
        Escreva em linguagem objetiva
        Evite ambiguidade ou termos vagos
        Use a estrutura: "O sistema deve..." ou "Deve ser possível..."
        Inclua critérios positivos e negativos, se aplicável
        """,
        description="Constrói critérios de aceitação claros, verificáveis e objetivos."
    )
    entrada_do_agente_gerador_criterios = f"Contexto: {contexto}\nContexto analisado: {contexto_analisado}\nUser stories: {user_stories}"
    # Executa o agente
    criterios = call_agent(gerador_criterios, entrada_do_agente_gerador_criterios)
    return criterios

In [77]:
##########################################
# --- Agente 4: Revisor Gerador de Cenário de Teste --- #
##########################################
def agente_gerador_cenario(contexto, contexto_analisado, user_stories,criterios):
    gerador_cenario = Agent(
        name="agente_gerador_cenario",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista de QA com foco em automação e comportamento orientado a testes (BDD).
        Sua missão é gerar cenários de teste comportamentais (Given / When / Then) com base na user story e nos critérios de aceitação fornecidos.
        Boas práticas que devem ser seguidas:
        Cada critério deve gerar pelo menos 1 cenário
        Use o formato Gherkin: Cenario, Dado que, Quando, Então
        Crie títulos de cenário descritivos
        Explore variações positivas e negativas, se possível
        Formate o texto de uma forma visual
        """,
        description="traduz os critérios de aceitação em cenários no estilo BDD (Behavior-Driven Development)"
    )
    entrada_do_agente_gerador_cenario = f"Contexto: {contexto}\nContexto analisado: {contexto_analisado}\nUser stories: {user_stories}\nCriterios: {criterios}"
    # Executa o agente
    cenarios = call_agent(gerador_cenario, entrada_do_agente_gerador_cenario)
    return cenarios

In [ ]:
print("🚀 Iniciando o Sistema de Especificação de User Stories com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
contexto = input("❓ Por favor, digite o CONTEXTO sobre o qual você quer especificar seus cards: ")

# Inserir lógica do sistema de agentes ################################################
if not contexto:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {contexto}")

    contexto_analisado = agente_analisador_contexto(contexto)
    print("\n--- 📝 Resultado do Agente 1 (Analisador de Contexto) ---\n")
    display(to_markdown(contexto_analisado))
    print("--------------------------------------------------------------")

    user_stories = agente_gerador_user_stories(contexto, contexto_analisado)
    print("\n--- 📝 Resultado do Agente 2 (Gerador de User Stories) ---\n")
    display(to_markdown(user_stories))
    print("--------------------------------------------------------------")

    criterios = agente_gerador_criterios(contexto, contexto_analisado, user_stories)
    print("\n--- 📝 Resultado do Agente 3 (Gerador de Critérios de Aceitação) ---\n")
    display(to_markdown(criterios))
    print("--------------------------------------------------------------")

    cenarios = agente_gerador_cenario(contexto, contexto_analisado, user_stories,criterios)
    print("\n--- 📝 Resultado do Agente 4 (Gerador de Cenários de Teste) ---\n")
    display(to_markdown(cenarios))
    print("--------------------------------------------------------------")